<a href="https://colab.research.google.com/github/adundulis/stable-diffusion/blob/main/lora_stable_diffusion_wrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LoRA stable diffusion

In [ ]:
#@markdown # Setup LoRA, Dependencies and Mount Google Drive

from IPython.utils import capture
import sys
import os
import time

SESSION_NAME = "" #@param{type: 'string'}

if SESSION_NAME == "":
  raise SystemExit('SESSION_NAME should not be empty.')

print('Mounting Google Drive...')
with capture.capture_output() as cap:
  from google.colab import drive
  drive.mount('/content/gdrive')

print('Installing LoRA and Dependencies...')
with capture.capture_output() as cap:
  !git clone https://github.com/cloneofsimo/lora.git && sed -i 's/functools.cache/functools.lru_cache(maxsize=None)/g' /content/lora/lora_diffusion/xformers_utils.py && pip install /content/lora
  !pip install accelerate bitsandbytes

print('Intialising Session Workspace...')
WORKSPACE='/content/gdrive/MyDrive/lora-stable-difussion'
SESSION_DIR = WORKSPACE + '/sessions/' + SESSION_NAME
IMAGES_DIR = SESSION_DIR + '/images'
CAPTIONS_DIR = SESSION_DIR + '/captions'
MODELS_DIR = SESSION_DIR + '/models'

with capture.capture_output() as cap:
  %cd /content
  if not os.path.exists(str(SESSION_DIR)):
    %mkdir -p $SESSION_DIR
    time.sleep(1)
  if not os.path.exists(str(IMAGES_DIR)):
    %mkdir -p $IMAGES_DIR
    time.sleep(1)
  if not os.path.exists(str(CAPTIONS_DIR)):
    %mkdir -p $CAPTIONS_DIR
    time.sleep(1)
  if not os.path.exists(str(MODELS_DIR)):
    %mkdir -p $MODELS_DIR
    time.sleep(1)

print('\033[92m' + 'Initialisation Successful' + '\033[0m')

In [ ]:
#@markdown # Upload Images and Captions